Covid-19 Data analysis for Austria

What we will do today:
* Pie plot
* Bar plot
* plotting on a map
* Line plot

The data is taken from here https://github.com/statistikat/coronaDAT.git

The map data for Austria is taken from here: https://www.data.gv.at/katalog/dataset/stat_gliederung-osterreichs-in-gemeinden14f53


The official data was finally published and can be found here: https://www.data.gv.at/covid-19/

In [ ]:
from IPython.display import clear_output

Install the libraries geopandas and gitpython

In [ ]:
!pip install geopandas
!pip install gitpython
clear_output()

Import all needed libraries

In [ ]:
import geopandas
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import urllib.request
import pathlib

from zipfile import ZipFile
import git
import os

% matplotlib inline

Clone the Git Repository which contains the Covid Data

The data was originally scraped from https://info.gesundheitsministerium.at/ before the layout of the dashboard was completely changed. Meanwhile the data is officially available here: https://www.data.gv.at/covid-19/

In [ ]:
os.mkdir("/content/covid-data")

repo = git.Repo.clone_from("https://github.com/statistikat/coronaDAT.git", "/content/covid-data", no_checkout=True)
repo.git.checkout("6aaadc8e4db5707dce148e34014f0e5324375f84") # 23.04.2021

In [ ]:
# Create a ZipFile Object and load our data in it
with ZipFile('/content/covid-data/archive/20210417/data/20210417_140201_orig_csv_ages.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('/content/covid-data/latest')

Have a look at the downloaded data. How is the structure? What kind of data does it contain?

Start with [PiePlot](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.pie.html) of gender data


In [ ]:
#load "CovidFaelle_Altersgruppe.csv" into the variable "agegroup"

agegroup = pd.read_csv("/content/covid-data/latest/CovidFaelle_Altersgruppe.csv", sep=";")

In [ ]:
#use the .head() function to look at the first few lines in the dataframe

agegroup.head()

In [ ]:
#select only the rows were Bundesland is "Österreich"

agegroup = agegroup[agegroup['Bundesland'] == "Österreich"]

In [ ]:
#calculate the sum of all male and female people who are currently sick (column "Anzahl" for the amount, column "Geschlecht" for the gender)

sum_m = sum(agegroup[agegroup['Geschlecht'] == "M"]['Anzahl'])
sum_f = sum(agegroup[agegroup['Geschlecht'] == "W"]['Anzahl'])

#store the data in a new dataframe, so that we can visualize it easily later on

gender = pd.DataFrame([["m", sum_m], ["f", sum_f]], columns=['gender', 'freq'])

In [ ]:
# use the .plot.pie() function
plot = gender.plot.pie(y='freq', figsize=(5, 5), autopct='%1.0f%%', pctdistance=0.3, labeldistance=1.2, labels=gender['gender'])

[PiePlot](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.pie.html) or [BarPlot](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.bar.html#matplotlib.axes.Axes.bar) for age data?

In [ ]:
#create a new dataframe named "age"

age = pd.DataFrame()

# create and fill columns for "agegroup", "female", "male" and "sum"
age['agegroup'] = list(dict.fromkeys(list(agegroup['Altersgruppe'])))
age['male'] = list(agegroup[agegroup['Geschlecht'] == "M"]['Anzahl'])
age['female'] = list(agegroup[agegroup['Geschlecht'] == "W"]['Anzahl'])

age['sum'] = age['male'] + age['female']

#add columns for "female_inhabitants", "male_inhabitants", "sum_inhabitants" and "sum_norm"
age['female_inhabitants'] = list(agegroup[agegroup['Geschlecht'] == "W"]['AnzEinwohner'])
age['male_inhabitants'] = list(agegroup[agegroup['Geschlecht'] == "M"]['AnzEinwohner'])
age['sum_inhabitants'] = age['male_inhabitants'] + age['female_inhabitants']
age['sum_norm'] = (age['male'] + age['female']) / age['sum_inhabitants']



In [ ]:
age.head()

In [ ]:
# use the .plot.pie() function
plot = age.plot.pie(y='sum', figsize=(5, 5), autopct='%1.0f%%', pctdistance=0.5, labeldistance=1.2, labels=age['agegroup'])

In [ ]:
# use the .plot.bar() function on column "sum"
plot = age.plot.bar(x='agegroup', y='sum', rot=40)

In [ ]:
# use the .plot.bar() function on column "sum_norm"
plot = age.plot.bar(x='agegroup', y='sum_norm', rot=40)

In [ ]:
# use the .plot.bar() function to show the features "female" and "male" for each "agegroup"
age.plot.bar(x='agegroup', y=['male', 'female'], rot=40)

Show the data for the single districts on a map

In [ ]:
# load "CovidFaelle_GKZ.csv" into the variable "cov_district"
cov_district = pd.read_csv("/content/covid-data/latest/CovidFaelle_GKZ.csv", sep=";")

In [ ]:
cov_district.head()

Download the Shapefile for Austria. It's a file that describes the borders of states and villages in Austria.

In [ ]:
#https://www.data.gv.at/katalog/dataset/stat_gliederung-osterreichs-in-gemeinden14f53
url = "http://data.statistik.gv.at/data/OGDEXT_GEM_1_STATISTIK_AUSTRIA_20200101.zip"
urllib.request.urlretrieve(url, '/content/Shapefile.zip')

In [ ]:
# Create a ZipFile Object and load sample.zip in it
with ZipFile('/content/Shapefile.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('Shapefile')

In [ ]:
# load the .shp file into the variable "austria"
austria = geopandas.read_file('/content/Shapefile/STATISTIK_AUSTRIA_GEM_20200101Polygon.shp', encoding='utf-8')

Look at the shapefile dataframe, plot it

In [ ]:
austria.head()

In [ ]:
austria.plot()

In [ ]:
austria[austria.name == "Linz"].plot()

Combine region data with cov data

In [ ]:
# implement an algorithm: we need a list of the first 3 numbers of the ID.
# there's one exception: whenever the Bezirk is in vienna, we have to take
# the id for vienna, which is 900.

In [ ]:
ids = []
for id in austria['id']:
  id = id[:-2]
  if id[0] == '9':
    ids.append(900)
  else:
    ids.append(int(id))

In [ ]:
austria['GKZ'] = ids # district_id

In [ ]:
# use the .merge() function to add the cov_district dataframe to the austria dataframe

austria = austria.merge(cov_district, on='GKZ', how="left").fillna(0)
austria.head()

In [ ]:
#plot the austria map with cases (Anzahl) as values
austria.plot(column='Anzahl', legend=True)

Show the relative cases 

In [ ]:
# calculate the relative cases, store them in the column 'relative_cases'
austria['relative_cases'] = austria['Anzahl'] / austria['AnzEinwohner']

In [ ]:
# plot the relative cases on the austria map
plt = austria.plot(column='relative_cases', legend=True)

[Lineplot](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html) of general data

In [ ]:
# load "CovidFaelle_Timeline.csv" into the variable "general"
# load "CovidFallzahlen.csv" into the variable "casenumbers"
general = pd.read_csv("/content/covid-data/latest/CovidFaelle_Timeline.csv", sep=";")
casenumbers = pd.read_csv("/content/covid-data/latest/CovidFallzahlen.csv", sep=";")

In [ ]:
# for both dataframes, reduce them to the values that refer to whole austria (BundeslandID = 10)
general = general[general['BundeslandID'] == 10]
casenumbers = casenumbers[casenumbers['BundeslandID'] == 10]

In [ ]:
casenumbers = casenumbers.rename(columns={'MeldeDatum': 'Time'})

In [ ]:
# use the pd.merge() function to merge the two dataframes together based on the "Time" value
# store it in a new dataframe "all_numbers"

all_numbers = pd.merge(general, casenumbers, on=['Time'], how='left').fillna(0)
all_numbers.head()

In [ ]:
# call the .plot.line() function (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.line.html)
lines = all_numbers.plot.line(x="Time", y=['AnzahlFaelle', 'FZHosp', 'FZICU', 'AnzahlTotTaeglich'], rot=45)

In [ ]:
lines = all_numbers.plot.line(subplots=True, x="Time", y=['AnzahlFaelle', 'FZHosp', 'FZICU', 'AnzahlTotTaeglich'])

Lineplot including healthy people

In [ ]:
# add column "AnzahlAktuell" and calculate the casenumbers
all_numbers['AnzahlAktuell'] = all_numbers['AnzahlFaelleSum'] - all_numbers['AnzahlGeheiltSum'] - all_numbers['AnzahlTotSum']

In [ ]:
# plot again the lineplot with more features
lines = all_numbers.plot.line(x="Time", y=['AnzahlFaelleSum', 'AnzahlGeheiltSum', 'AnzahlAktuell', 'FZHosp', 'FZICU', 'AnzahlTotSum'])